In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import tree
from sklearn import preprocessing


In [7]:
data=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx',sheet_name=1)
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [8]:
data=data.drop(columns=['ZIP Code','ID'])
data.drop_duplicates(inplace=True) 
data.head(2)

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0


In [10]:
data.isnull().sum()

Age                   0
Experience            0
Income                0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

**SPLITTING DATASET:**

In [18]:
y=df.iloc[:,9:10]
y.head(4)

,Personal Loan
0,0
1,0
2,0
3,0


In [20]:
x=df.drop(columns=["Personal Loan"])
x.head(4)


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0


In [22]:
tree_model=tree.DecisionTreeClassifier()
tree_model.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [24]:
tree_model.score(x,y)


1.0

This seems to be overfitted as accuracy score is 100%

RANDOM FOREST:

In [25]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)


In [28]:
rf_model.fit(x,y)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
rf_model.oob_score_

0.9856

**FINDING IMPORTANT VARIABLES:**

In [31]:
a = ['Age', 'Experience', 'Income','Family', 'CCAvg','Education', 'Mortgage', 'Securities Account','CD Account', 
     'Online', 'CreditCard']
for feature,imp in zip(a,rf_model.feature_importances_):
    print(feature,"\t\t :",imp)

Age 		 : 0.04407854190175194
Experience 		 : 0.0384068384837221
Income 		 : 0.03774178376244204
Family 		 : 0.32043475238531294
CCAvg 		 : 0.04176671309587588
Education 		 : 0.0841759057103513
Mortgage 		 : 0.17009356646109353
Securities Account 		 : 0.14389922121679585
CD Account 		 : 0.042338649956404184
Online 		 : 0.005331757048490783
CreditCard 		 : 0.05470037658326689


**From above we can infere that the important  independent variables are:<br>
Income, CCAvg, Education, Family**

**SPLIITING ACCORDING TO IMPORTANT VALUES**

In [32]:
imp = df[['Income','CCAvg','Education','Family']]
imp.head()

,Income,CCAvg,Education,Family
0,49,1.6,1,4
1,34,1.5,1,3
2,11,1.0,1,1
3,100,2.7,2,1
4,45,1.0,2,4


DECISION TREE:

In [33]:
model = tree.DecisionTreeClassifier(max_depth=12)
model.fit(imp,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [36]:
impf=["Income","CCAvg","Education","Family"]
from sklearn.tree import export_text
r=export_text(model,feature_names=impf)
print(r)

|--- Income <= 113.50
|   |--- CCAvg <= 2.95
|   |   |--- Income <= 106.50
|   |   |   |--- class: 0
|   |   |--- Income >  106.50
|   |   |   |--- Family <= 3.50
|   |   |   |   |--- Education <= 1.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- Education >  1.50
|   |   |   |   |   |--- CCAvg <= 1.65
|   |   |   |   |   |   |--- Family <= 1.50
|   |   |   |   |   |   |   |--- CCAvg <= 1.05
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- CCAvg >  1.05
|   |   |   |   |   |   |   |   |--- Income <= 112.00
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |--- Income >  112.00
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- Family >  1.50
|   |   |   |   |   |   |   |--- Income <= 111.50
|   |   |   |   |   |   |   |   |--- CCAvg <= 1.35
|   |   |   |   |   |   |   |   |   |--- Income <= 108.50
|   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |   |

In [34]:
model.score(imp,y)

0.9994